In [ ]:
# !pip install chess

In [2]:
import numpy as np
import pandas as pd
import chess
import chess.pgn
from ChessFunctions import get_encoded_board, flatten_board
from tqdm import tqdm
import dask
import dask.dataframe as dd
import dask.array as da
from dask import delayed

In [3]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:55183 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [4]:
# client.close()

In [5]:
f = open('lichess_elite_2020-06.pgn')

In [6]:
%%time 
my_list = []
Y_labels = []
for i in tqdm(range(100)):
# while True:
    game = chess.pgn.read_game(f)
    if game is None:
        break  # end of file
    my_list.append(game)

100%|██████████| 100/100 [00:00<00:00, 335.08it/s]

CPU times: user 282 ms, sys: 17.1 ms, total: 299 ms
Wall time: 302 ms


In [7]:
# def flatten_board(x):
#     return(str(x).replace(' ','').replace('\n',''))

In [8]:
# def get_encoded_board(x):
#     '''Must pass a flattened board'''
#     num_squares = range(64)
    
#     #Black Player Feature Vectors
#     b_rook  = [1 if x[i]=='r' else 0 for i in num_squares]
#     b_night = [1 if x[i]=='n' else 0 for i in num_squares]
#     b_bish  = [1 if x[i]=='b' else 0 for i in num_squares]
#     b_queen = [1 if x[i]=='q' else 0 for i in num_squares]
#     b_king  = [1 if x[i]=='k' else 0 for i in num_squares]
#     b_pawn  = [1 if x[i]=='p' else 0 for i in num_squares]
#     b_value = [(sum(b_rook)*5 + sum(b_night)*3 + sum(b_bish)*3
#            + sum(b_queen)*9 + sum(b_pawn)*1
#           )]
    
#     #White Player Feature Vectors
#     w_rook  = [1 if x[i]=='R' else 0 for i in num_squares]
#     w_night = [1 if x[i]=='N' else 0 for i in num_squares]
#     w_bish  = [1 if x[i]=='B' else 0 for i in num_squares]
#     w_queen = [1 if x[i]=='Q' else 0 for i in num_squares]
#     w_king  = [1 if x[i]=='K' else 0 for i in num_squares]
#     w_pawn  = [1 if x[i]=='P' else 0 for i in num_squares]
#     w_value = [(sum(w_rook)*5 + sum(w_night)*3 + sum(w_bish)*3
#            + sum(w_queen)*9 + sum(w_pawn)*1
#           )]

#     feature_list = (b_rook+b_night+b_bish+b_queen
#                 +b_king+b_pawn+w_rook+w_night+w_bish
#                 +w_queen+w_king+w_pawn+b_value+w_value)
    
# #     feature_list = dd.from_array(np.append(feature_list,[w_value,b_value]))
    
# #     feature_list.extend[w_value,b_value]


# #     feature_list = (b_rook+b_night+b_bish+b_queen
# #                 +b_king+b_pawn+w_rook+w_night+w_bish
# #                 +w_queen+w_king+w_pawn)
    
#     return(feature_list)

In [9]:
# def collect_game_states(games):
#     allgame_df = pd.DataFrame()
#     for game in games:
#         board = game.board() 
#         board_state = get_encoded_board(board)
#         for move in game.mainline_moves():
#             board.push(move)
#             board_state = np.row_stack((board_state,get_encoded_board(board)))
#         df = pd.DataFrame(board_state)
#         df['gameid'] = game.headers["LichessURL"]
#         allgame_df = pd.concat([allgame_df,df])
#     ddf = dd.from_pandas(allgame_df,npartitions=1)
#     return ddf

In [10]:
%%time
allgame_df = pd.DataFrame()
for game in tqdm(my_list):
    if game.headers['Termination']=='Normal':
        board = game.board() 
        board_state = [flatten_board(board)]
        for move in game.mainline_moves():
            board.push(move)
            board_state = np.row_stack((board_state,flatten_board(board)))
        df = pd.DataFrame(board_state)
        df['gameid'] = game.headers["LichessURL"]
        df['result'] = (np.where
                        (game.headers["Result"]=="1-0",'w',
                         np.where(game.headers["Result"]=="0-1",'l','d')
                        )
                       )
        df = df[3:]
        allgame_df = pd.concat([allgame_df,df])
    else:
        next

100%|██████████| 100/100 [00:00<00:00, 203.43it/s]

CPU times: user 480 ms, sys: 27.9 ms, total: 508 ms
Wall time: 493 ms


In [11]:
ddf = dd.from_pandas(allgame_df,npartitions=100000)

In [12]:
ddf = client.persist(ddf)

In [13]:
ddf['features'] = ddf[0].apply(get_encoded_board,meta=('I'))

In [14]:
ddf.compute()

,0,gameid,result,features
3,rnbqkbnrpppp.ppp....p..............PP............,https://lichess.org/kuUOsOML,d,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,rnbqkb.rpppppppp.....n............PP.............,https://lichess.org/MhgO70rO,w,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,rnbqkbnrpppp.ppp....p..............PP............,https://lichess.org/59fq1TbD,l,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,rnbqkb.rpppppppp.....n......P....................,https://lichess.org/00hqujG8,l,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,rnbqkbnrpppp.ppp....p..............PP............,https://lichess.org/OWFZIcDF,w,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
154,...........k.....K...............q...............,https://lichess.org/WGDO7qrJ,l,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
155,...........k....K................q...............,https://lichess.org/WGDO7qrJ,l,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
156,..........k.....K................q...............,https://lichess.org/WGDO7qrJ,l,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
157,........K.k......................q...............,https://lichess.org/WGDO7qrJ,l,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
def to_series(df):
    return df.features.apply(pd.Series, 1).stack().reset_index(level=1, drop=True)

In [16]:
# da_X = da.array(ddf.map_partitions(func))
# da_X.compute().shape

In [17]:
feature_array = np.array(ddf.map_partitions(to_series).compute()).reshape(-1,770)

In [18]:
X = feature_array
Y_labels = allgame_df['result']
print("X: " + str(X.shape))
print("Y: " + str(Y_labels.shape))

X: (5088, 770)
Y: (5088,)


In [19]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(
    X, Y_labels, stratify=Y_labels, train_size = 0.9
    ,random_state=1
)

CPU times: user 228 ms, sys: 36.6 ms, total: 265 ms
Wall time: 296 ms


In [20]:
X_train.shape, X_test.shape

((4579, 770), (509, 770))

In [29]:
param_grid = {'hidden_layer_sizes':[(25,25,25,10,10,10,5,5,5)]
              ,'tol':[0.00001]
              ,'max_iter':[600]
              ,'learning_rate_init':[0.0001]
             }
mlp = MLPClassifier(early_stopping=True)
clf = GridSearchCV(mlp, param_grid, n_jobs=-1)
# clf = MLPClassifier(hidden_layer_sizes=(400),random_state=1
#                     ,early_stopping=True,tol=.00001,max_iter=300).fit(X_train, y_train)
clf.fit(X_train, y_train)

GridSearchCV(estimator=MLPClassifier(early_stopping=True), n_jobs=-1,
             param_grid={'hidden_layer_sizes': [(25, 25, 25, 10, 10, 10, 5, 5,
                                                 5)],
                         'learning_rate_init': [0.0001], 'max_iter': [600],
                         'tol': [1e-05]})

In [30]:
print(clf.score(X_test, y_test))
clf.best_estimator_

0.39096267190569745


MLPClassifier(early_stopping=True,
              hidden_layer_sizes=(25, 25, 25, 10, 10, 10, 5, 5, 5),
              learning_rate_init=0.0001, max_iter=600, tol=1e-05)

In [23]:
# clf.predict(X_test)

In [24]:
from joblib import dump, load
dump(clf, 'filename.joblib')  

['filename.joblib']

In [25]:
clf = load('filename.joblib') 

In [26]:
# import torch

ModuleNotFoundError: No module named 'torch'

In [ ]:
# class Feedforward(torch.nn.Module):
#         def __init__(self, input_size, hidden_size):
#             super(Feedforward, self).__init__()
#             self.input_size = input_size
#             self.hidden_size  = hidden_size
#             self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
#             self.relu = torch.nn.ReLU()
#             self.fc2 = torch.nn.Linear(self.hidden_size, 1)
#             self.sigmoid = torch.nn.Sigmoid()
#         def forward(self, x):
#             hidden = self.fc1(x)
#             relu = self.relu(hidden)
#             output = self.fc2(relu)
#             output = self.sigmoid(output)
#             return output

In [ ]:
# model = Feedforward(770, 100)
# criterion = torch.nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
# import tensorflow as tf
# X_test_tensor = tf.convert_to_tensor(X_test)

In [ ]:
# model.eval()
# y_pred = model(X_test)
# before_train = criterion(y_pred.squeeze(), y_test)
# print('Test loss before training' , before_train.item())
